In [1]:
!nvidia-smi

Thu May  2 19:04:32 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.14              Driver Version: 550.54.14      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA RTX A5000               Off |   00000000:41:00.0 Off |                  Off |
| 30%   44C    P8             18W /  230W |       0MiB /  24564MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

torch.cuda.is_initialized()

False

In [3]:
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import ImageFolder
from datasets import load_dataset

resolution = 256

augmentations = transforms.Compose(
    [
        transforms.Resize((256, 256)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.5], [0.5]),
    ]
)

# augmentations = transforms.Compose(
#     [
#         transforms.Resize(resolution, interpolation=transforms.InterpolationMode.BILINEAR),
#         transforms.RandomCrop(resolution),
#         transforms.Lambda(lambda x: x),
#         transforms.ToTensor(),
#         transforms.Normalize([0.5], [0.5]),
#     ]
# )

def transform_images(examples):
    images = [augmentations(image.convert("RGB")) for image in examples["image"]]
    return {"input": images}

train_batch_size = 4
train_dataset_path = "ddpm-data/summer2winter_yosemite/train"
eval_dataset_path = "ddpm-data/summer2winter_yosemite/test"
train_ds = load_dataset("imagefolder", data_dir=train_dataset_path, split="train")
eval_ds = load_dataset("imagefolder", data_dir=eval_dataset_path, split="train")
print(f"Train Dataset size: {len(train_ds)}")
print(f"Eval Dataset size: {len(eval_ds)}")
train_ds.set_transform(transform_images)
eval_ds.set_transform(transform_images)
train_dataloader = torch.utils.data.DataLoader(
    train_ds, batch_size=train_batch_size, shuffle=True, num_workers=0
)
eval_dataloader = torch.utils.data.DataLoader(
    eval_ds, batch_size=train_batch_size, shuffle=True, num_workers=0
)

/nfshomes/skarki/scratch/miniconda3/envs/llm/lib/python3.9/site-packages/transformers/utils/generic.py:485: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


Resolving data files:   0%|          | 0/2193 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/547 [00:00<?, ?it/s]

Train Dataset size: 2193
Eval Dataset size: 547


In [4]:
torch.cuda.is_initialized()

False

In [5]:
# def forward(model, x):
#     # z_mu, z_var = Q(X)
#     # z = sample_z(z_mu, z_var)
#     # X_sample = P(z)

#     # # Loss
#     # recon_loss = nn.binary_cross_entropy(X_sample, X, size_average=False) / mb_size
#     # kl_loss = torch.mean(0.5 * torch.sum(torch.exp(z_var) + z_mu**2 - 1. - z_var, 1))
#     # loss = recon_loss + kl_loss

#     # posterior = model.encode(x).latent_dist
#     # print(posterior)
#     # if sample_posterior:
#     #     z = posterior.sample(generator=generator)
#     # else:
#     #     z = posterior.mode()
#     # dec = self.decode(z).sample

#     # if not return_dict:
#     #     return (dec,)

#     # return DecoderOutput(sample=dec)


#     posterior = model.encode(x).latent_dist
#     z = posterior.mode()
#     # z = posterior.sample()
    
#     x_sample = model.decode(z).sample
#     # Loss
#     recon_loss = F.mse_loss(x_sample, x)
#     # kl_loss = torch.mean(0.5 * torch.sum(torch.exp(z_var) + z_mu**2 - 1. - z_var, 1))
#     kl_loss = posterior.kl()
#     loss = recon_loss + kl_loss
#     return x_sample,  loss.mean(), recon_loss.mean(), kl_loss.mean()

In [6]:
# def forwardV2(model, x):
#     # z_mu, z_var = Q(X)
#     # z = sample_z(z_mu, z_var)
#     # X_sample = P(z)

#     # # Loss
#     # recon_loss = nn.binary_cross_entropy(X_sample, X, size_average=False) / mb_size
#     # kl_loss = torch.mean(0.5 * torch.sum(torch.exp(z_var) + z_mu**2 - 1. - z_var, 1))
#     # loss = recon_loss + kl_loss

#     # posterior = model.encode(x).latent_dist
#     # print(posterior)
#     # if sample_posterior:
#     #     z = posterior.sample(generator=generator)
#     # else:
#     #     z = posterior.mode()
#     # dec = self.decode(z).sample

#     # if not return_dict:
#     #     return (dec,)

#     # return DecoderOutput(sample=dec)


#     posterior = model.encode(x).latent_dist
#     z = posterior.mode()
#     # z = posterior.sample()
    
#     x_sample = model.decode(z).sample
#     # print(x_sample)
#     # Loss
#     # recon_loss = F.mse_loss(x_sample, x)
    
#     recon_loss = 0.5 * F.mse_loss(
#         x_sample.reshape(x.shape[0], -1),
#         x.reshape(x.shape[0], -1),
#         reduction="none",
#     ).sum(dim=-1)
#     # kl_loss = torch.mean(0.5 * torch.sum(torch.exp(z_var) + z_mu**2 - 1. - z_var, 1))
#     # kl_loss = posterior.kl()
#     # loss = recon_loss + kl_loss



#     # return 0.5 * torch.sum(
#     #                 torch.pow(self.mean, 2) + self.var - 1.0 - self.logvar,
#     #                 dim=[1, 2, 3],
#     #             )


#     # print(posterior.logvar.shape, posterior.mean.shape)
#     KLD = 0.5 * torch.sum(
#         torch.pow(posterior.mean, 2) + posterior.var - 1.0 - posterior.logvar,
#         dim=[1, 2, 3],
#     )

#     loss = (recon_loss + KLD).mean(dim=0)
#     return x_sample, loss , recon_loss.mean(dim=0), KLD.mean(dim=0)

In [7]:
# batch = next(iter(train_dataloader))

In [8]:
# out = forwardV2(model, batch["input"].to(device))
# out[1:]

In [9]:
# out[-1]

In [10]:
# import matplotlib.pyplot as plt

# _x = out[0][0].detach().cpu()
# plt.imshow(_x.permute(1, 2, 0))

In [11]:
# def show_img(img_tensor):
#     img_tensor = (img_tensor - img_tensor.min()) / (img_tensor.max() - img_tensor.min())
#     img_tensor = torch.clamp(img_tensor, 0, 1) * 255
    
#     # Convert to numpy and display using matplotlib
#     image = img_tensor.numpy().transpose(1, 2, 0)  # Assuming the format is CxHxW, convert to HxWxC
#     plt.imshow(image)
#     plt.axis('off')  # Turn off axis numbers and ticks
#     plt.show()
#     return img_tensor

In [12]:
# show_img(_x)

In [13]:
def train_loop(train_dataloader, eval_dataloader):
    import torch
    from tqdm.auto import tqdm, trange
    import os
    from diffusers.optimization import get_scheduler
    import lpips
    import random
    from torch.optim import Adam
    from torchvision.utils import make_grid
    from diffusers import AutoencoderKL
    from accelerate import Accelerator, InitProcessGroupKwargs
    from accelerate.utils import ProjectConfiguration
    from diffusers.utils import pt_to_pil, make_image_grid

        
    task_name = "summer-2-winter-vae-hf"
    num_epochs = 20
    gradient_accumulation_steps = 1

    model = AutoencoderKL.from_pretrained("CompVis/stable-diffusion-v1-4", subfolder="vae")
    optimizer = torch.optim.AdamW(
        model.parameters(),
        lr=1e-5,
        betas=(0.95, 0.999),
        weight_decay=1e-6,
        eps=1e-08,
    )


    lr_scheduler = get_scheduler(
        "cosine",
        optimizer=optimizer,
        num_warmup_steps=100 * gradient_accumulation_steps,
        num_training_steps=(len(train_dataloader) * num_epochs),
    )
    lpips_loss_fn = lpips.LPIPS(net="alex")

    accelerator = Accelerator(
        mixed_precision='fp16' ,
        gradient_accumulation_steps=1
    )

    if accelerator.is_main_process:
        accelerator.init_trackers("train_example")
        
    model,optimizer, train_dataloader, eval_dataloader, lr_scheduler, lpips_loss_fn = accelerator.prepare(
        model, optimizer, train_dataloader, eval_dataloader, lr_scheduler, lpips_loss_fn
    )
    step_count = 0
    # image_save_steps = 1
    image_save_steps = 100
    acc_steps = 4
    img_save_count = 0
    vae_autoencoder_ckpt_name = 'vqvae_autoencoder_ckpt.pth'

    kl_scale = 1e-6
    lpips_scale = 1e-1
    
    model = accelerator.unwrap_model(model)
    # Now you train the model
    for epoch in range(num_epochs):
        progress_bar = tqdm(total=len(train_dataloader), disable=not accelerator.is_local_main_process)
        progress_bar.set_description(f"Epoch {epoch}")
        losses = []
        mse_losses = []
        kl_losses = []
        lpips_losses = []
        optimizer.zero_grad()
        for batch in train_dataloader:
            step_count += 1
            x = batch["input"]
            with accelerator.accumulate(model):
                posterior = model.encode(x).latent_dist
                z = posterior.mode()
                pred = model.decode(z).sample
                kl_loss = posterior.kl().mean()
                mse_loss = F.mse_loss(pred, x, reduction="mean")
                lpips_loss = lpips_loss_fn(pred, x).mean()

                loss = (
                    mse_loss + lpips_scale * lpips_loss + kl_scale * kl_loss
                )
                
                accelerator.backward(loss)
                accelerator.clip_grad_norm_(model.parameters(), 1.0)
                # if step_count % acc_steps == 0:
                optimizer.step()
                optimizer.zero_grad()
                lr_scheduler.step()

                losses.append(loss.item())
                mse_losses.append(mse_loss.item())
                kl_losses.append(kl_loss.item())
                lpips_losses.append(lpips_loss.item())

                progress_bar.set_postfix(
                    loss=loss.item(), 
                    mse_loss=mse_loss.item(), 
                    kl_loss=kl_loss.item(),
                    lpips_loss=lpips_loss.item(),
                    step=step_count
                )
                progress_bar.update(1)
                # Image Saving Logic
                if step_count % image_save_steps == 0 or step_count == 1:
                    images = []
                    with torch.no_grad():
                        vae_model = accelerator.unwrap_model(model)
                        eval_batch = next(iter(eval_dataloader))
                        eval_x = eval_batch["input"]
                        reconstructions = vae_model(eval_x).sample
                        sample_size = min(8, eval_x.shape[0])
                        img_tensor = torch.cat([eval_x.cpu(), reconstructions.cpu()], axis=0)
                        images = pt_to_pil(eval_x.cpu()) + pt_to_pil( reconstructions.cpu())
                        # print(img_tensor.shape)
                        # print(sample_size)
                        # grid = make_grid(img_tensor, nrow=sample_size)
                        # print(grid.shape)
                        # img = transforms.ToPILImage()(grid)
                        img = make_image_grid(images, rows=2, cols=4)
                        if not os.path.exists(os.path.join(task_name,'vqvae_autoencoder_samples')):
                            os.mkdir(os.path.join(task_name, 'vqvae_autoencoder_samples'))
                        img.save(os.path.join(task_name,'vqvae_autoencoder_samples',
                                            'current_autoencoder_sample_{}.png'.format(img_save_count)))
                        img_save_count += 1
                        img.close()
                
                    
        optimizer.step()
        optimizer.zero_grad()
        avg_epoch_loss = sum(losses) / len(losses)
        avg_epoch_mse_loss = sum(mse_losses) / len(mse_losses)
        avg_epoch_kl_loss = sum(kl_losses) / len(kl_losses)
        avg_epoch_lpips_loss = sum(lpips_losses) / len(lpips_losses)
        progress_bar.set_postfix(
            avg_epoch_loss=avg_epoch_loss, 
            avg_epoch_mse_loss=avg_epoch_mse_loss, 
            avg_epoch_kl_loss=avg_epoch_kl_loss,
            avg_epoch_lpips_loss=avg_epoch_lpips_loss,
            step=step_count
        )
        model.save_pretrained(os.path.join(task_name, "model"))
        # torch.save(model.state_dict(), os.path.join(task_name, vae_autoencoder_ckpt_name))
    print('Done Training...')
    

In [14]:
from accelerate import notebook_launcher
args = (train_dataloader, eval_dataloader)

notebook_launcher(train_loop, args, num_processes=4)

Launching training on 4 GPUs.


/nfshomes/skarki/scratch/miniconda3/envs/llm/lib/python3.9/site-packages/transformers/utils/generic.py:342: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/nfshomes/skarki/scratch/miniconda3/envs/llm/lib/python3.9/site-packages/diffusers/utils/outputs.py:64: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
/nfshomes/skarki/scratch/miniconda3/envs/llm/lib/python3.9/site-packages/diffusers/utils/outputs.py:64: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
/nfshomes/skarki/scratch/miniconda3/envs/llm/lib/python3.9/site-packages/diffusers/utils/outputs.py:64: UserWarning: torch.utils._pytree._register_pytree_node is 

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]


/nfshomes/skarki/scratch/miniconda3/envs/llm/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/nfshomes/skarki/scratch/miniconda3/envs/llm/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(


/nfshomes/skarki/scratch/miniconda3/envs/llm/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/nfshomes/skarki/scratch/miniconda3/envs/llm/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/nfshomes/skarki/scratch/miniconda3/envs/llm/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMA

...

  0%|          | 0/138 [00:00<?, ?it/s]

  0%|          | 0/138 [00:00<?, ?it/s]

  0%|          | 0/138 [00:00<?, ?it/s]

  0%|          | 0/138 [00:00<?, ?it/s]

KeyboardInterrupt: 